In [2]:
from transformers import pipeline

In [3]:
prompt = """Instruct:  Using the information in the context, answer the question as concisely and faithfully as possible.If the context does not provide enough information to answer confidently, answer based on the most likely interpretation from the given text.

Context:

Document 0:. As of 26 January 2020, the still ongoing outbreak had resulted in 2066 (618 of them are in Wuhan) confirmed cases and 56 (45 of them were in Wuhan) deaths in mainland China [4] , and sporadic cases exported from Wuhan were reported in Thailand, Japan, Republic of Korea, Hong Kong, Taiwan, Australia, and the United States, please see the World Health Organization (WHO) news release via https://www.who.int/csr/don/en/ from 14 to 21 January 2020
Document 1:The first three cases detected were reported in France on 24 January 2020 and had onset of symptoms on 17, 19 and 23 January respectively [10] . The first death was reported on 15 February in France. As at 21 February, nine countries had reported cases ( Figure) : Belgium (1), Finland (1), France (12), Germany (16), Italy (3), Russia (2), Spain (2), Sweden (1) and the UK (9 -not included further).
Document 2:. Nonetheless, the virus spread from Wuhan to other cities in China and outside the country. By February 4, 2020, a total of 23 locations outside mainland China reported cases, 22 of which reported imported cases; Spain reported a case caused by secondary transmission (3).
Document 3:066 (618 of them are in Wuhan) confirmed cases and 56 (45 of them were in Wuhan) deaths in mainland China [4] , and sporadic cases exported from Wuhan were reported in Thailand, Japan, Republic of Korea, Hong Kong, Taiwan, Australia, and the United States, please see the World Health Organization (W
Document 4:As at 09:00 on 21 February, few COVID-19 cases had been detected in Europe compared with Asia. However the situation is rapidly developing, with a large outbreak recently identified in northern Italy, with transmission in several municipalities and at least two deaths [18] . As at 5 March 2020, there are 4,250 cases including 113 deaths reported among 38 countries in the WHO European region [19] .

Question:
As of 26 January 2020, what countries had sporadic cases?

Output:"""

## phi original

In [3]:
generator = pipeline("text-generation", model="microsoft/phi-2")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [4]:
output = generator(
    prompt,
    max_new_tokens=120,
    num_beams=5,
    early_stopping=True,
    length_penalty=0.1,
    do_sample=False,  # Desactiva sampling, activa búsqueda determinista
    return_full_text=False 
)

print(output[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



As of 26 January 2020, Thailand, Japan, Republic of Korea, Hong Kong, Taiwan, Australia, and the United States had sporadic cases.



In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed
from peft import PeftModel, PeftConfig
def load_model_auto(device="cuda", model_name=None, lora_adapter_path=None):
    if lora_adapter_path:
        print(f"Loading base model from LoRA adapter: {lora_adapter_path}")
        config = PeftConfig.from_pretrained(lora_adapter_path)
        model = AutoModelForCausalLM.from_pretrained(
            config.base_model_name_or_path,
            torch_dtype="auto",
            trust_remote_code=True,
        ).to(device)
        model = PeftModel.from_pretrained(model, lora_adapter_path).to(device)
        model = model.merge_and_unload()
        tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, trust_remote_code=True, padding_side="left")
    elif model_name:
        print(f"Loading base model: {model_name}")
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype="auto",
            trust_remote_code=True,
        ).to(device)
        tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, padding_side="left")
    else:
        raise ValueError("You must provide either a model_name or a lora_adapter_path.")

    # tokenizer.pad_token = tokenizer.eos_token
    # if tokenizer.pad_token_id is None:
    #     tokenizer.pad_token_id = tokenizer.eos_token_id

    

    return model, tokenizer

In [11]:
#model, tokenizer = load_model_auto('cuda', None, "../models/covid/adapters/best_phi_2_rag_covid_k3_5e_10bs")
model, tokenizer = load_model_auto('cuda', None, "../models/covid/adapters/phi_2_rag_covid_k3_5e_10bs/checkpoint-160")

Loading base model from LoRA adapter: ../models/covid/adapters/phi_2_rag_covid_k3_5e_10bs/checkpoint-160


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
generator2 = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [13]:
output2 = generator2(
    prompt,
    max_new_tokens=120,
    num_beams=5,
    early_stopping=True,
    length_penalty=1,
    do_sample=False,  # Desactiva sampling, activa búsqueda determinista
    return_full_text=False 
)

print(output2[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Wuhan, Thailand, Japan, Republic of Korea, Hong Kong, Taiwan, Australia, and the United States, please see the World Health Organization (WHO) news release via https://www.who.int/csr/don/en/ from 14 to 21 January 2020. The first death was reported on 15 February in France. As at 21 February, nine countries had reported cases ( Figure) : Belgium (1), Finland (1), France (12), Germany (16), Italy (3), Russia (2), Spain (2), Sweden (1) and the UK (


In [15]:
from transformers import AutoTokenizer

# Cargar el tokenizer de Phi-2
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")

# Texto de ejemplo
text = "¿Cuál es la capital de Francia?"

# Tokenizar
tokens = tokenizer.tokenize(text)
input_ids = tokenizer.encode(text, add_special_tokens=True)

# Mostrar tokens y sus IDs
print("Tokens:")
print(tokens)
print("\nToken IDs:")
print(input_ids)

# Mostrar tokens junto a sus IDs
print("\nTokens y sus IDs:")
for token, token_id in zip(tokens, input_ids):
    print(f"{token!r:20} -> {token_id}")


Tokens:
['Â', '¿', 'Cu', 'Ã¡', 'l', 'Ġes', 'Ġla', 'Ġcapital', 'Ġde', 'ĠFranc', 'ia', '?']

Token IDs:
[126, 123, 46141, 6557, 75, 1658, 8591, 3139, 390, 4682, 544, 30]

Tokens y sus IDs:
'Â'                  -> 126
'¿'                  -> 123
'Cu'                 -> 46141
'Ã¡'                 -> 6557
'l'                  -> 75
'Ġes'                -> 1658
'Ġla'                -> 8591
'Ġcapital'           -> 3139
'Ġde'                -> 390
'ĠFranc'             -> 4682
'ia'                 -> 544
'?'                  -> 30


In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")

# Obtener el token de fin de texto
eot_token = tokenizer.eos_token  # Esto debería ser '<|endoftext|>'
eot_token_id = tokenizer.eos_token_id

# Texto original
text = "¿Cuál es la capital de Francia?"

# Agregar manualmente el token de inicio y fin (en este caso, solo EOS si no hay BOS definido)
text_with_tokens = text + eot_token

# Tokenizar
input_ids = tokenizer.encode(text_with_tokens, add_special_tokens=True)

# Mostrar resultado
print("Texto con token final:", text_with_tokens)
print("Token IDs:", input_ids)
print("Tokens:", [tokenizer.decode([tid]) for tid in input_ids])


Texto con token final: ¿Cuál es la capital de Francia?<|endoftext|>
Token IDs: [126, 123, 46141, 6557, 75, 1658, 8591, 3139, 390, 4682, 544, 30, 50256]
Tokens: ['�', '�', 'Cu', 'á', 'l', ' es', ' la', ' capital', ' de', ' Franc', 'ia', '?', '<|endoftext|>']


In [26]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2",
                                          add_eos_token=True,
                                          trust_remote_code=True)
text = "¿Cuál es la capital de Francia?"
input_ids = tokenizer.encode(text, add_special_tokens=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.truncation_side = "left"
print("Token IDs:", input_ids)
print("Tokens:", [tokenizer.decode([tid]) for tid in input_ids])

Token IDs: [126, 123, 46141, 6557, 75, 1658, 8591, 3139, 390, 4682, 544, 30]
Tokens: ['�', '�', 'Cu', 'á', 'l', ' es', ' la', ' capital', ' de', ' Franc', 'ia', '?']


In [29]:
tokenizer = AutoTokenizer.from_pretrained(
    "microsoft/phi-2",
    padding_side="left",
    use_fast=False,
    trust_remote_code=True
)
#tokenizer.add_special_tokens({'pad_token': '<|padding|>'})
print(tokenizer.all_special_tokens)

['<|endoftext|>']


In [31]:
input_ids = tokenizer.encode(text)
tokenizer.truncation_side = "left"
print("Token IDs:", input_ids)
print("Tokens:", [tokenizer.decode([tid]) for tid in input_ids])

Token IDs: [126, 123, 46141, 6557, 75, 1658, 8591, 3139, 390, 4682, 544, 30]
Tokens: ['�', '�', 'Cu', 'á', 'l', ' es', ' la', ' capital', ' de', ' Franc', 'ia', '?']


In [8]:
from transformers import AutoTokenizer

# Cargar el tokenizer
model_name = "microsoft/phi-2"  # O cualquier otro modelo que estés usando
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(tokenizer.all_special_tokens)
# Secuencias de ejemplo
sequences = ["Esto es un ejemplo. <|endoftext|>", "Este es otro."]
tokenizer.pad_token = tokenizer.eos_token
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
# Tokenizar con padding
tokenized = tokenizer(sequences, padding=True, return_tensors="pt")

# Mostrar los tokens
print(tokenized['input_ids'])
# Convertir los IDs a tokens legibles
tokens = tokenizer.decode(tokenized['input_ids'][0])  # Primer ejemplo
print(tokens)

['<|endoftext|>']
tensor([[22362,    78,  1658,   555,   304,    73, 18856,    78,    13,   220,
         50256],
        [   36,  4169,  1658,   267, 23528,    13, 50256, 50256, 50256, 50256,
         50256]])
Esto es un ejemplo. <|endoftext|>


In [10]:
tokens = tokenizer.decode(tokenized['input_ids'][1])
tokens

'Este es otro.<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>'